# Concours sur les prix de vente immobiliers

In [5]:
# Importer les bibliothèques nécessaires
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Charger les données
train_path = 'train.csv'  # Remplacez par le chemin correct
X_full = pd.read_csv(train_path, index_col='Id')

test_path = 'test.csv'  # Remplacez par le chemin correct
X_test_full = pd.read_csv(test_path, index_col='Id')

# Prétraitement initial
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full['SalePrice']
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Séparer les données d'entraînement et de validation
X_train_full, X_valid_full, y_train, y_valid = train_test_split(
    X_full, y, train_size=0.8, test_size=0.2, random_state=0
)

# Identifier les colonnes numériques et catégoriques
categorical_cols = [
    cname for cname in X_train_full.columns
    if X_train_full[cname].dtype == "object" and X_train_full[cname].nunique() < 10
]
numerical_cols = [
    cname for cname in X_train_full.columns
    if X_train_full[cname].dtype in ['int64', 'float64']
]

# Garder les colonnes sélectionnées
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# Définir les transformations pour les colonnes numériques et catégoriques
numerical_transformer = SimpleImputer(strategy='median')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combiner les transformations dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Définir le modèle
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Créer le pipeline
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Entraîner le pipeline
my_pipeline.fit(X_train, y_train)

# Faire des prédictions sur les données de validation
preds_valid = my_pipeline.predict(X_valid)
mae = mean_absolute_error(y_valid, preds_valid)
print(f'MAE sur les données de validation : {mae}')

# Faire des prédictions sur les données de test
preds_test = my_pipeline.predict(X_test)

# Préparer le fichier de soumission
output = pd.DataFrame({'Id': X_test.index, 'SalePrice': preds_test})
output.to_csv('submission4.csv', index=False)
print("Fichier de soumission créé : submission4.csv")


MAE sur les données de validation : 17514.986232876712
Fichier de soumission créé : submission4.csv
